# Find the times of each mat and camera record, align recordings

This goes through all the mat and camera data, extracting the time periods each one spans. It aligns the recordings, pairing those that overlap intervals.

It then writes those pairings to a csv file.

In [1]:
import numpy as np
import pandas as pd
import os
import datetime

In [2]:
def splitall(path):
    """Split a file path into all its component parts"""
    allparts = []
    while 1:
        parts = os.path.split(path)
        if parts[0] == path:  # sentinel for absolute paths
            allparts.insert(0, parts[0])
            break
        elif parts[1] == path: # sentinel for relative paths
            allparts.insert(0, parts[1])
            break
        else:
            path = parts[0]
            allparts.insert(0, parts[1])
    return allparts

In [3]:
def mat_date_reader(d):
    """Read the timestamp in a mat csv file"""
    try:
        return datetime.datetime.strptime(d, '%Y-%m-%dT%H:%M:%S.%f%z')
    except ValueError:
        return datetime.datetime.strptime(d, '%Y-%m-%dT%H:%M:%S%z')

In [4]:
def camera_date_reader(d):
    try:
        return datetime.datetime.strptime(d, 'frame_%Y-%m-%dT%H_%M_%S.%f.csv')
    except ValueError:
        return datetime.datetime.strptime(d, 'frame_%Y-%m-%dT%H_%M_%S.csv')

In [56]:
times_mat = {}
for root, dirs, files in os.walk(os.path.join('data', 'mat_data')):
    if 'test' not in root:
        for file in files:
            if file.endswith('.csv'):
                mframe = pd.read_csv(os.path.join(root, file), 
                         converters={'Timestamp': mat_date_reader},
                        index_col='Frame')
                times_mat[(mframe.iloc[0]['Timestamp'].to_pydatetime().replace(tzinfo=None, microsecond=0), 
                           mframe.iloc[-1]['Timestamp'].to_pydatetime().replace(tzinfo=None, microsecond=0))
                            ] = root
times_mat

{(datetime.datetime(2024, 7, 18, 11, 4),
  datetime.datetime(2024, 7, 18, 11, 4, 6)): 'data/mat_data/Blaine/Falls PM/BLFPM06',
 (datetime.datetime(2024, 7, 18, 11, 27, 21),
  datetime.datetime(2024, 7, 18, 11, 27, 26)): 'data/mat_data/Blaine/Falls PM/BLFPM14',
 (datetime.datetime(2024, 7, 18, 11, 37, 28),
  datetime.datetime(2024, 7, 18, 11, 37, 32)): 'data/mat_data/Blaine/Falls PM/BLFPM27',
 (datetime.datetime(2024, 7, 18, 11, 26, 37),
  datetime.datetime(2024, 7, 18, 11, 26, 46)): 'data/mat_data/Blaine/Falls PM/BLFPM13',
 (datetime.datetime(2024, 7, 18, 11, 0, 42),
  datetime.datetime(2024, 7, 18, 11, 0, 51)): 'data/mat_data/Blaine/Falls PM/BLFPM02',
 (datetime.datetime(2024, 7, 18, 11, 31, 9),
  datetime.datetime(2024, 7, 18, 11, 31, 14)): 'data/mat_data/Blaine/Falls PM/BLFPM19',
 (datetime.datetime(2024, 7, 18, 11, 3, 14),
  datetime.datetime(2024, 7, 18, 11, 3, 24)): 'data/mat_data/Blaine/Falls PM/BLFPM05',
 (datetime.datetime(2024, 7, 18, 11, 45, 27),
  datetime.datetime(2024, 7,

In [64]:
times_camera = {}
for root, dirs, files in os.walk(os.path.join('data', 'camera_data')):
    if 'test' not in root:
        if dirs == []:
            fs = [f for f in sorted(files) if f.startswith('frame_2024')]
            if fs:
                times_camera[(camera_date_reader(fs[0]).replace(microsecond=0), 
                              camera_date_reader(fs[-1]).replace(microsecond=0))
                                ] = root
times_camera

{(datetime.datetime(2024, 7, 18, 11, 32, 32),
  datetime.datetime(2024, 7, 18, 11, 32, 42)): 'data/camera_data/Blaine/Falls IR/BLIRF21',
 (datetime.datetime(2024, 7, 18, 11, 27, 19),
  datetime.datetime(2024, 7, 18, 11, 27, 28)): 'data/camera_data/Blaine/Falls IR/BLIRF14',
 (datetime.datetime(2024, 7, 18, 11, 47, 48),
  datetime.datetime(2024, 7, 18, 11, 47, 57)): 'data/camera_data/Blaine/Falls IR/BLIRF36',
 (datetime.datetime(2024, 7, 18, 10, 56, 25),
  datetime.datetime(2024, 7, 18, 10, 56, 36)): 'data/camera_data/Blaine/Falls IR/BLIRF01',
 (datetime.datetime(2024, 7, 18, 11, 33, 17),
  datetime.datetime(2024, 7, 18, 11, 33, 26)): 'data/camera_data/Blaine/Falls IR/BLIRF22',
 (datetime.datetime(2024, 7, 18, 11, 42, 25),
  datetime.datetime(2024, 7, 18, 11, 42, 35)): 'data/camera_data/Blaine/Falls IR/BLIRF30',
 (datetime.datetime(2024, 7, 18, 11, 28, 17),
  datetime.datetime(2024, 7, 18, 11, 28, 26)): 'data/camera_data/Blaine/Falls IR/BLIRF15',
 (datetime.datetime(2024, 7, 18, 11, 3, 1

In [58]:
len(times_mat), len(times_camera)

(624, 575)

In [54]:
def i_before(i1, i2): return i1[1] < i2[0]
def i_disjoint(i1, i2): return (i_before(i1, i2) or i_before(i2, i1))
def i_overlaps(i1, i2): return not i_disjoint(i1, i2)

In [65]:
pairings = []
sorted_mat_times = list(sorted(times_mat))
sorted_camera_times = list(sorted(times_camera))

while sorted_mat_times and sorted_camera_times:
    m = sorted_mat_times[0]
    c = sorted_camera_times[0]
    if i_before(m, c):
        pairings += [(m, None)]
        sorted_mat_times = sorted_mat_times[1:]
    elif i_before(c, m):
        pairings += [(None, c)]
        sorted_camera_times = sorted_camera_times[1:]
    else:
        pairings += [(m, c)]
        sorted_mat_times = sorted_mat_times[1:]
        sorted_camera_times = sorted_camera_times[1:]

trailing_mats = [(m, None) for m in sorted_mat_times]
trailing_cameras = [(None, c) for c in sorted_camera_times]

pairings = pairings + trailing_mats + trailing_cameras
pairings

[((datetime.datetime(2024, 6, 11, 9, 53, 5),
   datetime.datetime(2024, 6, 11, 9, 53, 27)),
  (datetime.datetime(2024, 6, 11, 9, 53, 8),
   datetime.datetime(2024, 6, 11, 9, 53, 24))),
 ((datetime.datetime(2024, 6, 11, 9, 55, 31),
   datetime.datetime(2024, 6, 11, 9, 55, 59)),
  (datetime.datetime(2024, 6, 11, 9, 55, 44),
   datetime.datetime(2024, 6, 11, 9, 55, 55))),
 ((datetime.datetime(2024, 6, 11, 9, 56, 58),
   datetime.datetime(2024, 6, 11, 9, 57, 20)),
  (datetime.datetime(2024, 6, 11, 9, 57, 9),
   datetime.datetime(2024, 6, 11, 9, 57, 16))),
 ((datetime.datetime(2024, 6, 11, 9, 59, 36),
   datetime.datetime(2024, 6, 11, 10, 0, 4)),
  (datetime.datetime(2024, 6, 11, 9, 59, 47),
   datetime.datetime(2024, 6, 11, 10, 0, 1))),
 (None,
  (datetime.datetime(2024, 6, 11, 10, 1),
   datetime.datetime(2024, 6, 11, 10, 1, 12))),
 ((datetime.datetime(2024, 6, 11, 10, 4, 31),
   datetime.datetime(2024, 6, 11, 10, 4, 52)),
  (datetime.datetime(2024, 6, 11, 10, 4, 34),
   datetime.datetime

In [72]:
with open('paired_times.csv', 'w') as f:
    f.write("mat_file,mat_start,mat_end,mat_index,camera_index,camera_start,camera_end,camera_file\n")
    for m, c in pairings:
        if m:
            mn = ''.join(c for c in os.path.split(times_mat[m])[1] if c.isnumeric())
            m_out = f'"{times_mat[m]}","{m[0].isoformat()}","{m[1].isoformat()}",{mn}'
        else:
            m_out=',,,'
        if c:
            cn = ''.join(c for c in os.path.split(times_camera[c])[1] if c.isnumeric())
            c_out = f'{cn},"{c[0].isoformat()}","{c[1].isoformat()}","{times_camera[c]}"'
        else:
            c_out = ',,,'
        f.write(f'{m_out},{c_out}\n')